# Introduction

This is the demo how to use AI Bridge [Ben](https://github.com/lorserker/ben) and [redeal](https://github.com/anntzer/redeal) compare Leading for 3NTs

* http://www.rpbridge.net/4g00.htm
* book [Winning Notrump Leads](https://www.amazon.com/Winning-Notrump-Leads-David-Bird/dp/1554947596) - redeal uses the idea from this book
* AI Ben models: https://huggingface.co/AIBridgeEngine

# Preparation

both needs [dds](https://github.com/dds-bridge/dds) library, here gives the hacked solution without recompiling.

It shall work smoothly in [colab](https://colab.research.google.com/), verified in 2023.11.19, which is ubuntu jammy with sudo permission


In [12]:
# install ai ben library & redeal library
!pip install --upgrade git+https://github.com/larrycai/ben-lite # install ben python package, not released, hacked
!pip install --upgrade git+https://github.com/anntzer/redeal

  Cloning https://github.com/larrycai/ben-lite to /tmp/pip-req-build-vrsmni0i
  Running command git clone --filter=blob:none --quiet https://github.com/larrycai/ben-lite /tmp/pip-req-build-vrsmni0i
  Resolved https://github.com/larrycai/ben-lite to commit 67c584295264f555d1dc912daee7f0ce10a3b76b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/anntzer/redeal to /tmp/pip-req-build-m7sbjdnn
  Running command git clone --filter=blob:none --quiet https://github.com/anntzer/redeal /tmp/pip-req-build-m7sbjdnn
  Resolved https://github.com/anntzer/redeal to commit e2e81a477fd31ae548a340b5f0f380594d3d0ad6
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... don

In [13]:
!find ~/.local | grep redeal
!find ~/.local | grep ben
# !ldd ~/.local/lib/python3.10/site-packages/redeal/libdds.so
# may need to copy libdds.so to ben, this libdds remove mt-thread
#!cp ~/.local/lib/python3.10/site-packages/redeal/libdds.so !cp ~/.local/lib/python3.10/site-packages/ben/libdds.so

# Let's see the question

Chapter 1, Hand 3 from Book [Winning Notrump Leads](https://www.amazon.com/Winning-Notrump-Leads-David-Bird/dp/1554947596)

````
Bidding: (None all)
 West  North   East  South
                     1NT
 pass  3NT     all   pass

Sit in west: Which is the best lead

 South: 15-17 points, 5332, 4432, 4333
 North: 9-11 points, no 4 high cards
````

What would you lead from: ♠ J 8 6 ♥ 9 5 ♦ K Q 7 2 ♣ K J 7 2 ?

# Redeal's solution

Thomas Andrew's Deal is a deal generator: it outputs deals satisfying whatever conditions you specify, then estimate which is best by using dds

In [14]:
from redeal import *
from redeal import dds
from redeal.global_defs import Seat, Suit, Card, Rank, Strain
def accept(deal):
    north, south = deal.north, deal.south
    return (
        balanced(south) and       # south
        15 <= south.hcp <= 17 and # south
        len(north.spades) <4 and len(north.hearts) < 4 and # north
        9 <= north.hcp <= 11                               # north
    )

N_TRIES=500 # takes longer time if 5000, 500 is around
def nt_leader(predeal, del_lst=[], n_tries=N_TRIES):
    samples = []
    dealer = Deal.prepare(predeal)

    balanced = Shape("(4333)") + Shape("(4432)") + Shape("(5332)")

    simulation = OpeningLeadSim(accept, "3NS", imps)
    simulation.initial(dealer)

    found = 0
    for _ in range(1000*n_tries):
        if found >= n_tries:
            break
        deal = dealer()
        if not accept(deal):
            continue

        found += 1
        score_3NS = deal.dd_score('3NS')
        if score_3NS < 0:
            # it is down
            samples.append(deal)
        simulation.do(deal)

    # remove useless
    # check https://github.com/anntzer/redeal/blob/master/redeal/redeal.py#L623
    for card in del_lst:
        simulation.payoff.entries.remove(Card.from_str(card))

    print(f"Result of simulation: {found} deals")
    simulation.final(n_tries)
    return samples

predeal = {"W": "J86 95 KQ72 KJ72"}
print(predeal['W'])

cards_needs_removed=["SJ", "H5"]
samples = nt_leader(predeal,cards_needs_removed)
deal = samples[0]
Deal.set_str_style('long')
print("Check one sample")
print("3NT sample:", deal)

result=deal.dd_all_tricks("N" , "W")
score_3NS = deal.dd_score('3NS')


J86 95 KQ72 KJ72
Result of simulation: 500 deals
	♠8	♠6	♥9	♦K	♦7	♦2	♣K	♣J	♣7	♣2	
♠8		-0.13	-0.11	+0.19	+0.22	+0.61	+0.65	+0.60	+1.01	+0.96	+0.60	+0.55
		(0.05)	(0.06)	(0.12)	(0.11)	(0.15)	(0.14)	(0.14)	(0.13)	(0.14)	(0.12)	(0.12)
♠6	+0.13		+0.02	+0.32	+0.34	+0.74	+0.78	+0.72	+1.14	+1.08	+0.73	+0.67
	(0.05)		(0.04)	(0.11)	(0.11)	(0.15)	(0.14)	(0.14)	(0.13)	(0.14)	(0.12)	(0.12)
♥9	+0.11	-0.02		+0.30	+0.32	+0.71	+0.76	+0.70	+1.13	+1.07	+0.71	+0.66
	(0.06)	(0.04)		(0.10)	(0.11)	(0.14)	(0.13)	(0.13)	(0.13)	(0.13)	(0.12)	(0.12)
♦K	-0.19	-0.32	-0.30		+0.02	+0.42	+0.46	+0.41	+0.84	+0.79	+0.42	+0.36
	(0.12)	(0.11)	(0.10)		(0.02)	(0.12)	(0.13)	(0.13)	(0.12)	(0.12)	(0.11)	(0.11)
♦7	-0.22	-0.34	-0.32	-0.02		+0.40	+0.44	+0.38	+0.82	+0.76	+0.40	+0.34
	(0.11)	(0.11)	(0.11)	(0.02)		(0.12)	(0.13)	(0.13)	(0.12)	(0.12)	(0.11)	(0.11)
♦2	-0.61	-0.74	-0.71	-0.42	-0.40		+0.02	-0.03	+0.41	+0.35	+0.00	-0.06
	(0.15)	(0.15)	(0.14)	(0.12)	(0.12)		(0.10)	(0.10)	(0.15)	(0.15)	(0.14)	(0.14)
♣K	-0.65	-0.78	-0.76	-0.4

# AI Ben's solution

It uses AI Ben machine learning models to find the best leader


In [15]:
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

'/content/hfmodels'

In [26]:
from ben.nn.models import Models
from ben.bots import BotLead
from ben.sample import Sample
from ben.nn.leader import Leader
from ben.nn.bid_info import BidInfo
from ben.nn.bidder import Bidder
from ben.nn.lead_singledummy import LeadSingleDummy
lead = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead, sd_model, None, 0.1, 0.05,0.75, 0.66)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 5000, 5000, 128, 100, False)


In [27]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'J86.95.KQ72.KJ72'

# the auction goes:
auction = ['PAD_START','PAD_START','1N','PASS','3N','PASS', 'PASS', 'PASS']

# what to lead?

lead_bot = BotLead([True, True], hand, models, -1, -1, models.lead_threshold, sampler, False)
lead = lead_bot.find_opening_lead(auction)

Sorting by insta_score
Sx
0.5333621206066174


In [28]:
lead.card

S6

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [24]:
lead.to_dict()['candidates']

[{'card': 'Sx',
  'insta_score': 0.5334,
  'expected_tricks': 9.67,
  'p_make_contract': 0.73},
 {'card': 'Cx',
  'insta_score': 0.0759,
  'expected_tricks': 9.73,
  'p_make_contract': 0.74},
 {'card': 'Dx',
  'insta_score': 0.3701,
  'expected_tricks': 9.68,
  'p_make_contract': 0.74}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [25]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['A9x.K8.JT9x.Q98x Qxxxx.QJTxx.x.xx KT.Axxx.A8xx.ATx',
 'A9x.A8.T8xx.QT8x QTxx.QTxxx.x.9xx Kxx.KJxx.AJ9x.Ax',
 'Axx.QTx.A8x.QT9x Txx.J8xxx.JTx.8x KQ9x.AKx.9xx.Axx',
 'Kxx.Kxx.A8x.QT8x Q9x.JT8xx.xx.9xx ATxx.AQx.JT9x.Ax',
 '9xx.AJx.J9x.AQ9x Qxxx.8xxx.8xx.8x AKT.KQTx.ATx.Txx',
 'AQx.Tx.98xx.ATxx T9xx.KQxxx.xx.xx Kxx.AJ8x.AJT.Q98',
 'ATx.Qxx.Axx.QT8x Qxxx.T8xx.J9x.xx K9x.AKJx.T8x.A9x',
 'Axx.Txx.Tx.AQT8x Q9x.Q8xxx.8xx.xx KTxx.AKJ.AJ9x.9x',
 'AQx.xxx.JTx.AQ9x 9xxx.QT8x.9xx.8x KTx.AKJx.A8x.Txx',
 'Kx.AJx.AJ9x.8xxx T9xx.Q8xx.T8x.9x AQxx.KTxx.xx.AQT',
 'T9x.K8.A8xxx.Qxx Kxxx.QJTxxx..98x AQx.Axx.JT9x.ATx',
 'AKx.8xx.JTxx.Qxx 9xx.QJTxx.xx.T8x QTxx.AKx.A98.A9x',
 'AKQ.8x.9xx.T98xx T9xx.KJTxx.Tx.xx xxx.AQxx.AJ8x.AQ',
 'Kxx.JTx.AJ9xx.Tx T9xx.KQxxx.xx.8x AQx.A8x.T8.AQ9xx',
 'Kx.AQx.Txx.T8xxx QTxx.KJ8xxx.8x.x A9xx.Tx.AJ9x.AQ9',
 'Axx.Jxx.A98x.8xx T9xx.QT8x.Txx.Qx KQx.AKxx.Jx.AT9x',
 'Qxx.JT.AT9x.QTxx A9x.Q8xxx.xx.9xx KTxx.AKxx.J8x.A8',
 '9xx.KJT.AJx.T8xx Qxx.Qxxxx.98x.9x AKTx.A8x.Txx.AQx',
 'T9x.Kxx.